In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datetime

In [2]:
df_original = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)

In [45]:
df_features_bool = pd.read_csv('features_bool.csv', low_memory=False)

In [46]:
df_features_bool.head()

,person,busco_top_5_visitas,visito_mas_que_el_promedio,siempre_incrementando,inc_ultimo_mes,mismo_interes_ultimos_dos_meses
0,4886f805,0.0,False,0,1,0
1,ad93850f,1.0,True,0,1,0
2,0297fc1e,1.0,True,0,1,1
3,2d681dd8,1.0,False,0,1,0
4,cccea85e,0.0,True,0,1,0


In [47]:
df_ingresos_mensual = df_original[['person','timestamp','event']]

In [48]:
df_ingresos_mensual['mes'] = pd.to_datetime(df_ingresos_mensual['timestamp']).dt.month

C:\Users\juanp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
df_ultimo_mes_que_ingreso = df_ingresos_mensual.sort_values(by='mes', ascending=False).drop_duplicates(subset='person',keep='first')

In [50]:
df_ultimo_mes_que_ingreso['ingreso_el_ultimo_mes'] = df_ultimo_mes_que_ingreso['mes'] == 5

In [51]:
df_features_bool = df_features_bool.merge(df_ultimo_mes_que_ingreso[['person','ingreso_el_ultimo_mes']], on='person', how='left')

In [52]:
df_ingresos_mensual['semana'] = pd.to_datetime(df_ingresos_mensual['timestamp']).dt.week

C:\Users\juanp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [53]:
df_ingreso_ultima_quincena = df_ingresos_mensual.sort_values(by='semana', ascending=False).drop_duplicates(subset='person',keep='first')

In [54]:
df_ingreso_ultima_quincena['ingreso_ultima_quincena'] = df_ingreso_ultima_quincena['semana'] >= 21

In [55]:
df_features_bool = df_features_bool.merge(df_ingreso_ultima_quincena[['person','ingreso_ultima_quincena']],on='person',how='left')

In [57]:
df_features_bool.head()

,person,busco_top_5_visitas,visito_mas_que_el_promedio,siempre_incrementando,inc_ultimo_mes,mismo_interes_ultimos_dos_meses,ingreso_el_ultimo_mes,ingreso_ultima_quincena
0,4886f805,0.0,False,0,1,0,True,False
1,ad93850f,1.0,True,0,1,0,True,True
2,0297fc1e,1.0,True,0,1,1,True,True
3,2d681dd8,1.0,False,0,1,0,True,True
4,cccea85e,0.0,True,0,1,0,True,True


In [58]:
df_features_bool.to_csv(path_or_buf='features_bool.csv',index=False)